Aqui, começaremos a atacar o problema de sincronizar nossos arquivos de audio com os acordes escritos. Primeiramente, tentaremos, para uma dada música, determinar os acordes por compasso. Faremos isso tomando como base os arquivos em formato xml.

In [29]:
import pandas as pd
import numpy as np
import os
import xml.etree.ElementTree as ET
song_title = 'ItsOn'
tree = ET.parse('corpus/xml/{}.xml'.format(song_title))
root = tree.getroot()

In [30]:
part = root.findall('part')[0]
measures = part.findall('measure')
for measure in measures:
    chords = measure.findall('harmony')
    for chord in chords:
        try:
            note = chord.find('root/root-step').text

        except AttributeError: # se não achar a nota "root" do acorde, não tem acorde
            print('Measure: {}'.format(measure.attrib['number']), ' | ', 'No chords')
            continue
            
        if note:
            # alter é a alteração de um acorde (nem sempre existe): b ou #.
            alter = chord.find('root/root-alter')#.text
            
            # kind é o tipo do acorde (nem sempre existe): pode ser com 7a, 9a, 13a, por aí vai...
            kind = chord.find('kind')#.attrib['text']
            
            # bass é o baixo do acorde, quando diferente da tônica. Também nem sempre existe.
            bass = chord.find('bass/bass-step')#.text
            
            # bass_alter é e alteração do baixo, podendo ser b ou #. Também nem sempre existe.
            bass_alter = chord.find('bass/bass-alter')#.text
            
            attribs_dict = {'alter':alter, 
                            'kind':kind, 
                            'bass':bass, 
                            'bass_alter':bass_alter}

            for key, value in attribs_dict.items():
                if value is None:
                    attribs_dict[key] = ''
                elif key == 'kind':
                    attribs_dict[key] = value.attrib['text']
                else:
                    attribs_dict[key] = value.text
                    
            print('Measure: {}'.format(measure.attrib['number']), '|', 'Chord: ', note, 
                  attribs_dict['alter'], attribs_dict['kind'],'/',attribs_dict['bass'],attribs_dict['bass_alter'])
            

Measure: 1 | Chord:  F  min7 /  
Measure: 5 | Chord:  F  min7 /  
Measure: 6 | Chord:  C   / E 
Measure: 7 | Chord:  F  min7 / E -1
Measure: 8 | Chord:  B -1  / D 
Measure: 8 | Chord:  E -1  /  
Measure: 8 | Chord:  C   / E 
Measure: 9 | Chord:  B -1  / D 
Measure: 9 | Chord:  E -1  /  
Measure: 9 | Chord:  C   /  
Measure: 10 | Chord:  F  min7 /  
Measure: 11 | Chord:  B -1 9 /  
Measure: 12 | Chord:  E -1 9 /  
Measure: 13 | Chord:  D -1  / A -1
Measure: 13 | Chord:  A -1  /  
Measure: 13 | Chord:  G  min7 /  
Measure: 13 | Chord:  C  7 /  
Measure: 14 | Chord:  F  min7 /  
Measure: 15 | Chord:  B -1 9 /  
Measure: 16 | Chord:  E -1 9 /  
Measure: 17 | Chord:  D -1  / A -1
Measure: 17 | Chord:  A -1  /  
Measure: 17 | Chord:  G  min7 /  
Measure: 17 | Chord:  C  7 /  


Comparando com a partitura aberta no musescore parece fazer sentido, só preciso conseguir fazer com que mostre também os compassos em que não tem acorde (no caso da música que estamos usando como exemplo, os compassos 1 e 2).

Agora, vamos modificar ligeiramente o código acima para adicionar as informações em um dataframe.

In [31]:
df_teste = pd.DataFrame(columns=[song_title])
df_add = pd.DataFrame(columns=[song_title], 
                      index=[3], 
                      data='E -1 min7')

df_add_2 = pd.DataFrame(columns=[song_title], 
                      index=[3], 
                      data='D -1')

In [32]:
df_teste = df_teste.append(df_add)

In [33]:
df_teste.append(df_add_2)

,ItsOn
3,E -1 min7
3,D -1


In [34]:
df_harmony = pd.DataFrame(columns=[song_title])

part = root.findall('part')[0]
measures = part.findall('measure')

for measure in measures:
    chords = measure.findall('harmony')
    for chord in chords:
        try:
            note = chord.find('root/root-step').text

        except AttributeError: # se não achar a nota "root" do acorde, não tem acorde
            print('Measure: {}'.format(measure.attrib['number']), ' | ', 'No chords')
            continue
            
        if note:
            # alter é a alteração de um acorde (nem sempre existe): b ou #.
            alter = chord.find('root/root-alter')#.text
            
            # kind é o tipo do acorde (nem sempre existe): pode ser com 7a, 9a, 13a, por aí vai...
            kind = chord.find('kind')#.attrib['text']
            
            # bass é o baixo do acorde, quando diferente da tônica. Também nem sempre existe.
            bass = chord.find('bass/bass-step')#.text
            
            # bass_alter é e alteração do baixo, podendo ser b ou #. Também nem sempre existe.
            bass_alter = chord.find('bass/bass-alter')#.text
            
            attribs_dict = {'alter':alter, 
                            'kind':kind, 
                            'bass':bass, 
                            'bass_alter':bass_alter}

            for key, value in attribs_dict.items():
                if value is None:
                    attribs_dict[key] = ''
                elif key == 'kind':
                    attribs_dict[key] = value.attrib['text']
                else:
                    attribs_dict[key] = value.text
                    
                    
            measure_number = measure.attrib['number']
            chord = note + attribs_dict['alter'] + attribs_dict['kind']
            chord_bass = attribs_dict['bass']+attribs_dict['bass_alter']
            complete_chord = chord + '/' + chord_bass
            
            if complete_chord.endswith('/'):
                complete_chord = complete_chord[:-1]
            
            df_chord = pd.DataFrame(columns=[song_title], index=[measure_number], data=complete_chord)
            df_harmony = df_harmony.append(df_chord)
            
df_harmony.index.names = ['Measure']

In [35]:
df_harmony.index.unique()

Index(['1', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
       '17'],
      dtype='object', name='Measure')

In [36]:
df_harmony.index.value_counts()

13    4
17    4
9     3
8     3
14    1
12    1
15    1
10    1
16    1
11    1
7     1
1     1
6     1
5     1
Name: Measure, dtype: int64

In [37]:
df_harmony

,ItsOn
Measure,
1,Fmin7
5,Fmin7
6,C/E
7,Fmin7/E-1
8,B-1/D
8,E-1
8,C/E
9,B-1/D
9,E-1


Agora vamos procurar os momentos de mudança de acorde. Vasculhando os arquivos xml que temos em nosso corpus, pudemos ver que as informações sobre andamento (em bpm) e tipo de compasso encontram-se na seção do primeiro compasso. Segue:

#### Tipo de compasso

In [38]:
first_measure = measures[0]
cima = first_measure.find('attributes').find('time/beats').text
baixo = first_measure.find('attributes').find('time/beat-type').text

compasso = cima + '/' + baixo
compasso

'4/4'

#### Andamento

In [39]:
andamento = first_measure.find('sound').attrib['tempo']# + ' bpm'
andamento
# em bpm

'96'

#### Sync: Acorde + minutagem

In [40]:
df_harmony['Time'] = ''
df_harmony

,ItsOn,Time
Measure,,
1,Fmin7,
5,Fmin7,
6,C/E,
7,Fmin7/E-1,
8,B-1/D,
8,E-1,
8,C/E,
9,B-1/D,
9,E-1,


In [41]:
duracao_seminima = 60/int(andamento) # em segundos

Fazendo caso a caso pra construir a generalização:

In [42]:
# if int(baixo) == 2: # a unidade é a mínima
#     segs_compasso = 2 * int(cima) * duracao_seminima
    
# elif int(baixo) == 4: # a unidade é a seminínima
#     segs_compasso = 1 * int(cima) * duracao_seminima
    
# elif int(baixo) == 8: # a unidade é a colcheia
#     segs_compasso = (1/2) * int(cima) * duracao_seminima
    
# elif int(baixo) == 16: # a unidade é a semicolcheia
#     segs_compasso = (1/4) * int(cima) * duracao_seminima

Checar se a generalização abaixo tá certa:

In [43]:
segs_compasso = (4/int(baixo)) * int(cima) * duracao_seminima

In [44]:
df_harmony.index = df_harmony.index.astype(int)

In [45]:
for compasso in df_harmony.index.unique():
    acordes = df_harmony.loc[[compasso]]
    n_acordes = len(acordes)
    print(compasso, n_acordes)
    inicio_compasso = (int(compasso) - 1) * segs_compasso
    
    if n_acordes == 1:
        tempos = inicio_compasso
    
    else:        
        tempos = np.linspace(inicio_compasso,
                             inicio_compasso + segs_compasso,
                             n_acordes+1)
        tempos = tempos[:-1]
#     df_harmony.loc[compasso]['Time'] = tempos
    
    df_harmony.at[compasso, 'Time'] = tempos

1 1
5 1
6 1
7 1
8 3
9 3
10 1
11 1
12 1
13 4
14 1
15 1
16 1
17 4


In [46]:
df_harmony

,ItsOn,Time
Measure,,
1,Fmin7,0
5,Fmin7,10
6,C/E,12.5
7,Fmin7/E-1,15
8,B-1/D,17.5
8,E-1,18.3333
8,C/E,19.1667
9,B-1/D,20
9,E-1,20.8333


Botei pra tocar e as mudanças de acorde parecem estar fazendo sentido! 
Uma coisa que acabei de pensar é sobre os ritornellos... acho que desta maneira não os reconhecemos. Verificar depois.

## Lidando com ritornellos

In [47]:
part = root.findall('part')[0]
measures = part.findall('measure')
for measure in measures:
    barras = measure.findall('barline')
    for barra in barras:
        
        try:
            ritornellos = barra.findall('repeat')
            for rito in ritornellos:
                print('Ritornello', measure.attrib['number'], rito.attrib['direction'])
        except:
            continue
            
        try:
            casas = barra.findall('ending')
            for casa in casas:
                print('Casa', measure.attrib['number'], casa.attrib['type'])
        except: #AttributeError: #significa que não tem ritornello no compasso
#             print(measure.attrib['number'], 'Não tem rito nesse compasso')
            continue
            


Ritornello 5 forward
Casa 8 start
Ritornello 8 backward
Casa 8 stop
Casa 9 start
Casa 9 discontinue


#### Casas
O que descobri: elas vêm como um "ending" dentro de um "barline". O interessante é que, para construir uma casa, precisamos de duas estruturas desse tipo: uma pra começar a casa e outra pra terminar. Então, por exemplo, se uma casa começa e termina num mesmo compasso, teremos dois "barlines" com "ending" dentro; um com "type" igual a "start" e outro com "type" igual a "stop" ou "descontinue" (o que vi até agora).